In [39]:
from bpe import byte_pair_encoding
import pandas as pd

Data was retrieved from [kaggle](https://www.kaggle.com/datasets/kazanova/sentiment140/).

In [40]:
FILE_PATH = "Sentiment140/training.1600000.processed.noemoticon.csv"
df = pd.read_csv(FILE_PATH, header=None, encoding='latin-1')
header_names = ["target", "ids", "date", "flag", "user", "text"]
df.columns = header_names
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [44]:
byte_pair_encoding(df.head()["text"], 200)

defaultdict(int,
            {'@switchfoot</s>': 1,
             'http://twitpic.com/2y1zl</s>': 1,
             '-</s>': 1,
             'Awww,</s>': 1,
             "that's</s>": 1,
             'a</s>': 2,
             'bummer.</s>': 1,
             'You</s>': 1,
             'shoulda</s>': 1,
             'got</s>': 1,
             'David</s>': 1,
             'Carr</s>': 1,
             'of</s>': 2,
             'Third</s>': 1,
             'Day</s>': 1,
             'to</s>': 2,
             'do</s>': 1,
             'it.</s>': 1,
             ';D</s>': 1,
             'is</s>': 1,
             'upset</s>': 1,
             'that</s>': 1,
             'he</s>': 1,
             "can't</s>": 2,
             'update</s>': 1,
             'his</s>': 1,
             'Facebook</s>': 1,
             'by</s>': 1,
             'texting</s>': 1,
             'it...</s>': 1,
             'and</s>': 2,
             'might</s>': 1,
             'cry</s>': 1,
             'as</s>': 1,
         